In [1]:
import pandas as pd 
import altair as alt
import panel as pn
import numpy as np
from sklearn.linear_model import LogisticRegression
import pickle
from sqlalchemy import URL, create_engine, text
alt.data_transformers.disable_max_rows()

hostname = 'database-3.cluster-cgldqbnitk50.us-east-1.rds.amazonaws.com'
username = 'postgres'
password = 'cas_analytics'
database = 'postgres'

url_object = URL.create(
    "postgresql",
    username=username,
    password=password,  # plain (unescaped) text
    host=hostname,
    database=database,
)

engine = create_engine(url_object)

In [2]:
# cols = [
#     'Monthly Reporting Period',
#     'Loan Identifier',
#     'Reference Pool ID',
#     'Loan Identifier',
#     'Original Interest Rate',
#     'Current Interest Rate',
#     'Original UPB',
#     'Current Actual UPB',
#     'Loan Age',
#     'Original Loan to Value Ratio (LTV)',
#     'Original Combined Loan to Value Ratio (CLTV)',
#     'Borrower Credit Score At Issuance',
#     'Borrower Credit Score Current ',
#     'Debt-To-Income (DTI)',
#     'UPB at the Time of Removal',
#     'Repurchase Date',
#     'Zero Balance Code',
#     'Zero Balance Effective Date',
#     'Zero Balance Code Change Date',
#     'Scheduled Principal Current',
#     'Total Principal Current',
#     'Unscheduled Principal Current',
#     'Zip Code Short',
#     'Property State',
#     'Current Loan Delinquency Status'
# ]

cols = [
    'Loan Identifier',
    'Monthly Reporting Period',
    'Original Interest Rate',
    'Original UPB',
    'Original Loan to Value Ratio (LTV)',
    'Original Combined Loan to Value Ratio (CLTV)',
    'Borrower Credit Score At Issuance',
    'Debt-To-Income (DTI)',
    'Zip Code Short',
    'Property State',
    'Current Loan Delinquency Status'
]

query = f"""
WITH loan_stats AS (
    SELECT 
        "Loan Identifier",
        "Original Interest Rate",
        "Original UPB",
        "Original Loan to Value Ratio (LTV)",
        "Original Combined Loan to Value Ratio (CLTV)",
        "Borrower Credit Score At Issuance",
        "Debt-To-Income (DTI)",
        "Zip Code Short",
        ROUND(
            1.0*SUM(CASE WHEN "Current Loan Delinquency Status" > 0 THEN 1 ELSE 0 END)/COUNT("Monthly Reporting Period"),
        2
        ) AS pct_dlq_marks
    FROM Cas.ref  
    GROUP BY 
        "Loan Identifier",
        "Original Interest Rate",
        "Original UPB",
        "Original Loan to Value Ratio (LTV)",
        "Original Combined Loan to Value Ratio (CLTV)",
        "Borrower Credit Score At Issuance",
        "Debt-To-Income (DTI)",
        "Zip Code Short"
)

SELECT * FROM loan_stats
"""

df = pd.DataFrame(engine.connect().execute(text(query)))

In [3]:
df['dlq_flag'] = df.pct_dlq_marks > 0
df.pct_dlq_marks = df.pct_dlq_marks.astype(float)

In [4]:
df.fillna(-1, inplace=True)

Next Steps:
1. Build a logistic regression model to get the cofficients of all the variables
2. Input a loan's attributes, calculate the pct difference between each attribute and the average among loans in the same ZIP, then take a weighted average of the pct differences to determine the risk of the loan going delinquent and the biggest contributing factors

In [70]:
train_var = ['Original Interest Rate', 'Original UPB',
       'Original Loan to Value Ratio (LTV)',
       'Borrower Credit Score At Issuance', 'Debt-To-Income (DTI)']

test_var = 'dlq_flag'
X_train = df[train_var]
y_train = df[test_var]

# Create a Logistic Regression model
model = LogisticRegression()

# Train the model on the training data
model = model.fit(X_train, y_train)

In [78]:
weights = dict(zip(train_var, model.coef_[0]))

In [79]:
weights

{'Original Interest Rate': 0.0009107327515811359,
 'Original UPB': -5.756557287793569e-07,
 'Original Loan to Value Ratio (LTV)': -0.020989459579410397,
 'Borrower Credit Score At Issuance': -0.001414122982036496,
 'Debt-To-Income (DTI)': 0.0353275235736391}

In [82]:
with open('../outputs/weights.pkl', 'wb') as file:
    pickle.dump(weights, file)

In [5]:
with open('../outputs/weights.pkl', 'rb') as file:
    weights = pickle.load(file)